In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import re
import urllib.parse
import numpy as np
import math
from time import sleep
import random
import sqlite3
from sqlite3 import Error
import os
import random
import time
from IPython.display import display, Audio
import time
import re

In [220]:
df_3 = pd.DataFrame(columns=['title_persian','title_english','price','discount','grade','code','shabak',
                               'count_page','realese_year_sh','release_year_mi','exist_bolean','earliest_access',
                               'print_serie', 'publisher_name','category','ghate','cover', 'writer',
                               'translator','book_id','writer_id','translator_id','category_id','publisher_id'])

In [3]:
my_header = {"Accept-Language": "en-US,en;q=0.9",
           "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}

In [4]:
url = 'https://www.iranketab.ir/book'

In [5]:
response = requests.get(url, headers= my_header)
print(response.status_code)
from http.client import responses
responses[response.status_code]

200


'OK'

In [6]:
soup = BeautifulSoup(response.text, "html.parser")
links = soup.select('.product-name-title .product-item-link')
print(len(links))

20


In [7]:
link = [name.get('href') for name in  links]
new_link = ['https://www.iranketab.ir'+name for name in link]
new_link

['https://www.iranketab.ir/book/43-a-fraction-of-the-whole',
 'https://www.iranketab.ir/book/84-the-kite-runner',
 'https://www.iranketab.ir/book/132-harry-potter',
 'https://www.iranketab.ir/book/133-my-name-is-red',
 'https://www.iranketab.ir/book/146-accident-nocturne',
 'https://www.iranketab.ir/book/155-one-hundred-years-of-solitude',
 'https://www.iranketab.ir/book/202-the-unnamable',
 'https://www.iranketab.ir/book/204-watt',
 'https://www.iranketab.ir/book/223-conversation-in-the-cathedral',
 'https://www.iranketab.ir/book/242-strait-is-the-gate',
 'https://www.iranketab.ir/book/273-local-anaesthetic',
 'https://www.iranketab.ir/book/281-tortilla-flat',
 'https://www.iranketab.ir/book/302-roots',
 'https://www.iranketab.ir/book/358-headhunter',
 'https://www.iranketab.ir/book/380-the-age-of-reason',
 'https://www.iranketab.ir/book/396-the-counterfeiters',
 'https://www.iranketab.ir/book/398-walk-two-moons',
 'https://www.iranketab.ir/book/415-island-of-the-blue-dolphins',
 'htt

In [8]:
from persiantools.jdatetime import JalaliDate

def convert_to_jalali(date_string):
    day = int(''.join(filter(str.isdigit, date_string)))
    month = None
    if 'فروردین' in date_string:
        month = 1
    elif 'اردیبهشت' in date_string:
        month = 2
    elif 'خرداد' in date_string:
        month = 3
    elif 'تیر' in date_string:
        month = 4
    elif 'مرداد' in date_string:
        month = 5
    elif 'شهریور' in date_string:
        month = 6
    elif 'مهر' in date_string:
        month = 7
    elif 'آبان' in date_string:
        month = 8
    elif 'آذر' in date_string:
        month = 9
    elif 'دی' in date_string:
        month = 10
    elif 'بهمن' in date_string:
        month = 11
    elif 'اسفند' in date_string:
        month = 12
    else:
        raise ValueError('ماه نامعتبر است')

    jalali_date = JalaliDate(1402, month, day)
    new = jalali_date.strftime("%Y-%m-%d")
    return new

In [9]:
def mge_jnge(url_main):
    global main_df
    main_df = pd.DataFrame(columns=['title_persian','title_english','price','discount','grade','code','shabak',
                               'count_page','realese_year_sh','release_year_mi','exist_bolean','earliest_access',
                               'print_serie', 'publisher_name','category','ghate','cover', 'writer',
                               'translator','book_id','writer_id','translator_id','category_id','publisher_id'])
    response = requests.get(url_main, headers= my_header)
    print(response.status_code)
    from http.client import responses
    responses[response.status_code]
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.find_all('div',{'class':'col-md-9 col-sm-9'})
    print(len(rows))
    for i in rows:
        #dataframe
        df = pd.DataFrame(columns=['title_persian','title_english','price','discount','grade','code','shabak',
                               'count_page','realese_year_sh','release_year_mi','exist_bolean','earliest_access',
                               'print_serie', 'publisher_name','category','ghate','cover', 'writer',
                               'translator','book_id','writer_id','translator_id','category_id','publisher_id'])
        
        #title_persian
        try:
            title_persian = i.select('.product-name strong')[0].text
            title_persian= title_persian.replace('کتاب ' , '')
            df.loc[0,'title_persian'] = title_persian
        except:
            df.loc[0,'title_persian'] = np.nan
        
        #title_english
        try:
            title_english = i.select('.product-name-englishname')[0].text
            df.loc[0,'title_english'] = title_english
        except:
            df.loc[0,'title_english'] = np.nan
            
    
        #price
        price_each_row = i.select('.clearfix~ .clearfix+ .clearfix .price , .col-md-7 .price-special')[0].text
        price = int(price_each_row.replace(',',''))
        df.loc[0,'price'] = price
    
        #discount
        try:
            discount_each_row = i.select('.col-md-7 li:nth-child(1) div')[0].text
            temp_discount = re.findall(r'\d+', discount_each_row)[0]
            df.loc[0,'discount'] = temp_discount
        except:
            df.loc[0,'discount'] = np.nan
    
        #grade
        try:
            Grade_each_row = i.find_all('div', {'class':'my-rating'})
            attribute_grade = [x['data-rating'] for x in Grade_each_row if 'data-rating' in x.attrs]
            df.loc[0,'grade'] = float(attribute_grade[0])
        except:
            df.loc[0,'grade'] = np.nan

        #realese_year_sh
        try:
            realese_year_sh_each_row = i.select('tr:nth-child(6) .rtl')[0].text
            realese_year_sh_each_row = realese_year_sh_each_row.replace('\r\n','')
            realese_year_sh_each_row = realese_year_sh_each_row.replace(' ','')
            df.loc[0,'realese_year_sh'] = int(realese_year_sh_each_row)
        except:
            df.loc[0,'realese_year_sh'] = np.nan
        #exist_bolean
        try:
            exist_bolean_each_row = i.select('.icon-exit , .exists-book span')[0].text
            exist_bolean_each_row = exist_bolean_each_row.replace(' ','')
            if exist_bolean_each_row == 'موجود':
                df.loc[0,'exist_bolean'] = (True)
            else:
                df.loc[0,'exist_bolean'] = (False)
        except:
            df.loc[0,'exist_bolean'] = np.nan
        #publisher_name
        try:
            publisher_name_each_row = i.select('.prodoct-attribute-items:nth-child(1) a .prodoct-attribute-item')[0].text
            df.loc[0,'publisher_name'] = publisher_name_each_row
        except:
            df.loc[0,'publisher_name'] = np.nan
        #category
        try:
            category_each_row = soup.select('.product-tags-item')
            category = [name.text for name in category_each_row]
            df.loc[0,'category'] = category
        except:
            df.loc[0,'category'] = np.nan
        #writer
        try:
            writer_each_row = i.select('.prodoct-attribute-items+ .prodoct-attribute-items a .prodoct-attribute-item')
            writer_each_row = [name.text for name in writer_each_row]
            df.loc[0,'writer'] = writer_each_row
        except:
            df.loc[0,'writer'] = np.nan
        #translator 
        try:
            translator_each_row = i.select('.product-table .prodoct-attribute-item')
            translator_each_row = [name.text for name in translator_each_row]
            if len(translator_each_row) == 0 :
                df.loc[0,'translator'] = [np.nan]
            else:
                df.loc[0,'translator'] = translator_each_row
        except:
            df.loc[0,'translator'] = np.nan
            
        #book_id
        match_book_id= re.search(r'(\d+)', url_main)
        number_book_id = int(match_book_id.group(1))
        df.loc[0,'book_id'] = number_book_id
        
        #writer_id
        try:
            w_id = []
            writer_each_row = i.select('.prodoct-attribute-items+ .prodoct-attribute-items a')
            writer_each_row = [name.get('href') for name in writer_each_row]
            for sublist in writer_each_row:
                match_writer_id = re.search(r'(\d+)', sublist)
                number_writer_id = int(match_writer_id.group(1))
                w_id.append(number_writer_id)
            df.loc[0,'writer_id'] = w_id
        except:
            df.loc[0,'writer_id'] = np.nan
            
        #translator_id
        try:
            tr_id = []
            translator_id_each_rows = i.select('tr:nth-child(2) td+ td a')
            translator_id_each_row = [name.get('href') for name in translator_id_each_rows]
            for sublist in translator_id_each_row:
                match_translator_id = re.search(r'(\d+)', sublist)
                number_translator_id= int(match_translator_id.group(1))
                tr_id.append(number_translator_id)
            df.loc[0,'translator_id'] = tr_id
        except:
            translator_ids.append(np.nan)
            
        #category_id
        try:
            category_id_each_row = soup.select('.product-tags-item')
            category_id = [name.get('href') for name in category_id_each_row]
            category_ids = [re.findall(r'[0-9]+', name)[0] for name in category_id]
            df.loc[0,'category_id'] = category_ids
        except:
            df.loc[0,'category_id'] = np.nan
            
        #publisher_id
        try:
            pub_id = []
            publisher_id_each_rows = i.select('.prodoct-attribute-items:nth-child(1) a')
            publisher_id_each_row = [name.get('href') for name in publisher_id_each_rows]
            for sublist in publisher_id_each_row:
                match_publisher_id = re.search(r'(\d+)', sublist)
                number_publisher_id = int(match_publisher_id.group(1))
                pub_id.append(number_publisher_id)
            df.loc[0,'publisher_id'] = pub_id
        except:
            df.loc[0,'publisher_id'] = np.nan
            
        #table
        for j in i.select('.row .col-md-5 tr'):
            j = str(j.text)
            
            #code
            if 'کد کتاب' in j:
                temp_code = int(re.findall(r'\d+', j)[0])
                df.loc[0,'code'] = temp_code
                
            #shabak
            elif 'شابک' in j:
                temp_shabak = j.replace('شابک', '')
                temp_shabak = temp_shabak.replace(':', '')
                temp_shabak = temp_shabak.replace(' ', '')
                temp_shabak = temp_shabak.replace('\n', '')
                temp_shabak = temp_shabak.replace('\r', '')
                df.loc[0,'shabak'] = temp_shabak
                
            #ghate
            elif 'قطع' in j:
                temp_ghate = j.replace('قطع', '')
                temp_ghate = temp_ghate.replace(':', '')
                temp_ghate = temp_ghate.replace(' ', '')
                temp_ghate = temp_ghate.replace('\n', '')
                temp_ghate = temp_ghate.replace('\r', '')
                df.loc[0,'ghate'] = temp_ghate
                
            #counter
            elif 'تعداد صفحه' in j:
                temp_count = int(re.findall(r'\d+', j)[0])
                df.loc[0,'count_page'] =int(temp_count)
                
            #shamsi
            elif 'سال انتشار شمسی' in j:
                temp_shamsi = int(re.findall(r'\d+', j)[0])
                df.loc[0,'realese_year_sh'] = int(temp_shamsi)
                
            #miladi
            elif 'سال انتشار میلادی' in j:
                temp_miladi = int(re.findall(r'\d+', j)[0])
                df.loc[0,'release_year_mi'] = int(temp_miladi)
                
            #cover
            elif 'نوع جلد' in j:
                temp_cover = j.replace('نوع جلد', '')
                temp_cover = temp_cover.replace(':', '')
                temp_cover = temp_cover.replace(' ', '')
                temp_cover = temp_cover.replace('\n', '')
                temp_cover = temp_cover.replace('\r', '')
                df.loc[0,'cover'] = temp_cover
                
            #serie
            elif 'سری چاپ' in j:
                temp_serie = int(re.findall(r'\d+', j)[0])
                df.loc[0,'print_serie'] = int(temp_serie)
                
            #earliest
            elif 'زودترین زمان ارسال' in j:
                temp_earliest = j.replace('زودترین زمان ارسال', '')
                temp_earliest  = temp_earliest .replace(':', '')
                temp_earliest  = temp_earliest .replace(' ', '')
                temp_earliest  = temp_earliest .replace('\n', '')
                temp_earliest  = temp_earliest .replace('\r', '')
                try:
                    earliest_access_date = convert_to_jalali(temp_earliest)
                    df.loc[0,'earliest_access'] = earliest_access_date
                except:
                    df.loc[0,'earliest_access'] = np.nan
        main_df = pd.concat([main_df,df], axis=0)

In [221]:
#main
def mge_jnge(url_main):
    global main_df
    main_df = pd.DataFrame(columns=['title_persian','title_english','price','discount','grade','code','shabak',
                               'count_page','realese_year_sh','release_year_mi','exist_bolean','earliest_access',
                               'print_serie', 'publisher_name','category','ghate','cover', 'writer',
                               'translator','book_id','writer_id','translator_id','category_id','publisher_id'])
    response = requests.get(url_main, headers= my_header)
    print(response.status_code)
    from http.client import responses
    responses[response.status_code]
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.find_all('div',{'class':'col-md-9 col-sm-9'})
    print(len(rows))
    for i in rows:
        #dataframe
        df = pd.DataFrame(columns=['title_persian','title_english','price','discount','grade','code','shabak',
                               'count_page','realese_year_sh','release_year_mi','exist_bolean','earliest_access',
                               'print_serie', 'publisher_name','category','ghate','cover', 'writer',
                               'translator','book_id','writer_id','translator_id','category_id','publisher_id'])
        
        #title_persian
        try:
            title_persian = i.select('.product-name strong')[0].text
            title_persian= title_persian.replace('کتاب ' , '')
            df.loc[0,'title_persian'] = title_persian
        except:
            df.loc[0,'title_persian'] = np.nan
        
        #title_english
        try:
            title_english = i.select('.product-name-englishname')[0].text
            df.loc[0,'title_english'] = title_english
        except:
            df.loc[0,'title_english'] = np.nan
    
        #price
        try:
            price_each_row = i.find_all('span' , attrs={'class':'price'})[0].text
            price = int(price_each_row.replace(',',''))
            df.loc[0,'price'] = price
        except:
            df.loc[0,'price'] = np.nan
        #discount
        try:
            discount_each_row = i.select('.col-md-7 li:nth-child(1) div')[0].text
            temp_discount = re.findall(r'\d+', discount_each_row)[0]
            df.loc[0,'discount'] = temp_discount
        except:
            df.loc[0,'discount'] = np.nan
    
        #grade
        try:
            Grade_each_row = i.find_all('div', {'class':'my-rating'})
            attribute_grade = [x['data-rating'] for x in Grade_each_row if 'data-rating' in x.attrs]
            df.loc[0,'grade'] = float(attribute_grade[0])
        except:
            df.loc[0,'grade'] = np.nan

        #realese_year_sh
        try:
            realese_year_sh_each_row = i.select('tr:nth-child(6) .rtl')[0].text
            realese_year_sh_each_row = realese_year_sh_each_row.replace('\r\n','')
            realese_year_sh_each_row = realese_year_sh_each_row.replace(' ','')
            df.loc[0,'realese_year_sh'] = int(realese_year_sh_each_row)
        except:
            df.loc[0,'realese_year_sh'] = np.nan
        #exist_bolean
        try:
            exist_bolean_each_row = i.select('.icon-exit , .exists-book span')[0].text
            exist_bolean_each_row = exist_bolean_each_row.replace(' ','')
            if exist_bolean_each_row == 'موجود':
                df.loc[0,'exist_bolean'] = (True)
            else:
                df.loc[0,'exist_bolean'] = (False)
        except:
            df.loc[0,'exist_bolean'] = np.nan
        #publisher_name
        try:
            publisher_name_each_row = i.select('.prodoct-attribute-items:nth-child(1) a .prodoct-attribute-item')[0].text
            df.loc[0,'publisher_name'] = publisher_name_each_row
        except:
            df.loc[0,'publisher_name'] = np.nan
        #category
        try:
            category_each_row = soup.select('.product-tags-item')
            category = [name.text for name in category_each_row]
            df.loc[0,'category'] = category
        except:
            df.loc[0,'category'] = np.nan
        
        #writer
        ws = []
        try:
            for w_id in i.find_all('div',attrs={"class" : "row clearfix"}):
                span_tags_w_id = w_id.find_all('span' ,attrs={"itemprop" : "name"})
                ws.append(span_tags_w_id[0].text)
                if len(ws) == 0:
                    df.at[0,'writer'] = [np.nan]
                else:
                    df.at[0,'writer'] = ws
        except:
            df.at[0,'writer'] = [np.nan]
                        
        #translator 
        try:
            translator_each_row = i.select('.product-table .prodoct-attribute-item')
            translator_each_row = [name.text for name in translator_each_row]
            if len(translator_each_row) == 0:
                df.at[0,'translator'] = [np.nan]
            else:
                df.at[0,'translator'] = translator_each_row
        except:
            df.at[0,'translator'] = [np.nan]
            
        #book_id
        match_book_id= re.search(r'(\d+)', url_main)
        number_book_id = int(match_book_id.group(1))
        df.loc[0,'book_id'] = number_book_id
        
        #writer_id
        w_ids = []
        try:
            for w_id in i.find_all('div',attrs={"class" : "row clearfix"}):
                span_tags_w_id = w_id.find_all('span',attrs={"itemprop" : "name"})
                for tag in span_tags_w_id:
                    up = tag.find_parent('a')
                    if up.name == 'a':
                        match_writer_id = re.search(r'(\d+)', up.get('href'))
                        number_writer_ids= int(match_writer_id.group(1)) 
                        w_ids.append(number_writer_ids)
                if len(w_ids) == 0:
                    df.loc[0,'writer_id'] = [np.nan]
                else:
                    df.at[0,'writer_id'] = w_ids
        except:
            df.loc[0,'writer_id'] = [np.nan]
            
        #translator_id
        for j in i.select('tr:nth-child(2) td+ td'):
            tr_id = []
            span_tags = j.find_all('span')
            for tag in span_tags:
                    up = tag.parent
                    if up.name == 'a':
                        match_translator_id = re.search(r'(\d+)', up.get('href'))
                        number_translator_id= int(match_translator_id.group(1))
                        tr_id.append(number_translator_id)
                    else:
                        tr_id.append(np.nan)
        if len(tr_id) == 0:
            df.at[0,'translator_id'] = [np.nan]
        else: 
            df.at[0,'translator_id'] = tr_id
            
        #category_id
        try:
            category_id_each_row = soup.select('.product-tags-item')
            category_id = [name.get('href') for name in category_id_each_row]
            category_ids = [re.findall(r'[0-9]+', name)[0] for name in category_id]
            df.loc[0,'category_id'] = category_ids
        except:
            df.loc[0,'category_id'] = np.nan
            
        #publisher_id
        try:
            pub_id = []
            publisher_id_each_rows = i.select('.prodoct-attribute-items:nth-child(1) a')
            publisher_id_each_row = [name.get('href') for name in publisher_id_each_rows]
            for sublist in publisher_id_each_row:
                match_publisher_id = re.search(r'(\d+)', sublist)
                number_publisher_id = int(match_publisher_id.group(1))
                pub_id.append(number_publisher_id)
            df.loc[0,'publisher_id'] = pub_id
        except:
            df.loc[0,'publisher_id'] = np.nan
            
        #table
        for j in i.select('.row .col-md-5 tr'):
            j = str(j.text)
            
            #code
            if 'کد کتاب' in j:
                temp_code = int(re.findall(r'\d+', j)[0])
                df.loc[0,'code'] = temp_code
                
            #shabak
            elif 'شابک' in j:
                temp_shabak = j.replace('شابک', '')
                temp_shabak = temp_shabak.replace(':', '')
                temp_shabak = temp_shabak.replace(' ', '')
                temp_shabak = temp_shabak.replace('\n', '')
                temp_shabak = temp_shabak.replace('\r', '')
                df.loc[0,'shabak'] = temp_shabak
                
            #ghate
            elif 'قطع' in j:
                temp_ghate = j.replace('قطع', '')
                temp_ghate = temp_ghate.replace(':', '')
                temp_ghate = temp_ghate.replace(' ', '')
                temp_ghate = temp_ghate.replace('\n', '')
                temp_ghate = temp_ghate.replace('\r', '')
                df.loc[0,'ghate'] = temp_ghate
                
            #counter
            elif 'تعداد صفحه' in j:
                temp_count = int(re.findall(r'\d+', j)[0])
                df.loc[0,'count_page'] =int(temp_count)
                
            #shamsi
            elif 'سال انتشار شمسی' in j:
                temp_shamsi = int(re.findall(r'\d+', j)[0])
                df.loc[0,'realese_year_sh'] = int(temp_shamsi)
                
            #miladi
            elif 'سال انتشار میلادی' in j:
                temp_miladi = int(re.findall(r'\d+', j)[0])
                df.loc[0,'release_year_mi'] = int(temp_miladi)
                
            #cover
            elif 'نوع جلد' in j:
                temp_cover = j.replace('نوع جلد', '')
                temp_cover = temp_cover.replace(':', '')
                temp_cover = temp_cover.replace(' ', '')
                temp_cover = temp_cover.replace('\n', '')
                temp_cover = temp_cover.replace('\r', '')
                df.loc[0,'cover'] = temp_cover
                
            #serie
            elif 'سری چاپ' in j:
                temp_serie = int(re.findall(r'\d+', j)[0])
                df.loc[0,'print_serie'] = int(temp_serie)
                
            #earliest
            elif 'زودترین زمان ارسال' in j:
                temp_earliest = j.replace('زودترین زمان ارسال', '')
                temp_earliest  = temp_earliest .replace(':', '')
                temp_earliest  = temp_earliest .replace(' ', '')
                temp_earliest  = temp_earliest .replace('\n', '')
                temp_earliest  = temp_earliest .replace('\r', '')
                try:
                    earliest_access_date = convert_to_jalali(temp_earliest)
                    df.loc[0,'earliest_access'] = earliest_access_date
                except:
                    df.loc[0,'earliest_access'] = np.nan
        main_df = pd.concat([main_df,df], axis=0)

In [222]:
import pandas as pd
df_url = pd.read_csv('book_url.csv')

In [223]:
df_1 = df_url.iloc[:4000,:]
df_1

,book_url
0,https://www.iranketab.ir/book/110080-mutlulu%c...
1,https://www.iranketab.ir/book/32953-mantegh-e-...
2,https://www.iranketab.ir/book/4544-religion-vs...
3,https://www.iranketab.ir/book/92718-bia-ba-man...
4,https://www.iranketab.ir/book/84551-web-design
...,...
3995,https://www.iranketab.ir/book/92064-helping-yo...
3996,https://www.iranketab.ir/book/47719-the-wolf-w...
3997,https://www.iranketab.ir/book/14713-things
3998,https://www.iranketab.ir/book/78943-ashnaee-ba...


In [224]:
from IPython.display import Audio
sound_yes = './Applause1 -Sound Effects.mp3'
sound_no = './phone-beeps-sound.mp3'

In [225]:
import time
start = time.time()
try:
    for i in df_1['book_url']:
        print(i)
        mge_jnge(i)
        df_3 = pd.concat([df_3,main_df], axis=0)
        Audio(sound_yes ,autoplay=True)
except:
    Audio(sound_no ,autoplay=True)
    
end = time.time()
print(end - start)
Audio(sound_yes ,autoplay=True)

https://www.iranketab.ir/book/110080-mutlulu%c4%9fu-sende-bulan-senindir-%c3%b6tesi-misafir
200
1
https://www.iranketab.ir/book/32953-mantegh-e-heyrani
200
1
https://www.iranketab.ir/book/4544-religion-vs-religion
200
1
https://www.iranketab.ir/book/92718-bia-ba-man-bebin
200
1
https://www.iranketab.ir/book/84551-web-design
200
1
https://www.iranketab.ir/book/27109-morality-in-rumi-s-view
200
1
https://www.iranketab.ir/book/70847-book-of-my-mother
200
1
https://www.iranketab.ir/book/110094-aye-ba-kolah
200
1
https://www.iranketab.ir/book/94116-practical-ikigai
200
1
https://www.iranketab.ir/book/65193-az-banafsh-tond-ta
200
1
https://www.iranketab.ir/book/68520-social-network-analysis
200
1
https://www.iranketab.ir/book/73828-the-essence-of-christianity
200
1
https://www.iranketab.ir/book/82324-the-alloy-of-law
200
1
https://www.iranketab.ir/book/21036-arabic-fiction
200
1
https://www.iranketab.ir/book/19377-il-libro-della-visione
200
1
https://www.iranketab.ir/book/50901-dar-damane-ha

In [227]:
df_3

,title_persian,title_english,price,discount,grade,code,shabak,count_page,realese_year_sh,release_year_mi,...,category,ghate,cover,writer,translator,book_id,writer_id,translator_id,category_id,publisher_id
0,همسفر زندگی,Mutluluğu Sende Bulan Senindir Ötesi Misafir,119900,20,3.36,110080,978-6222208806,160,1402,2023,...,"[ ادبیات ترکیه , روانشناسی , دهه 2020 میلادی...",رقعی,شومیز,[هاکان منگوچ],[فائزه پورعلی],110080,[13315],[13316],"[151, 327, 801, 1291]",[1153]
0,منطق حیرانی,Mantegh-e Heyrani,170000,NaN,3.59,32953,978-6004901673,304,1401,NaN,...,"[ ادبیات واقع گرایانه , ادبیات ایران , حقوقی ]",رقعی,شومیز,[حسن جعفری تبار],[nan],32953,[15821],[nan],"[139, 437, 591]",[47]
0,مذهب علیه مذهب,Religion Vs. Religion,130000,30,3.79,4544,978-6005957884,392,1400,1971,...,"[ ادبیات معاصر , دهه 1970 میلادی , فلسفی , ...",رقعی,شومیز,[علی شریعتی],[nan],4544,[4324],[nan],"[105, 171, 291, 353, 437]",[1492]
0,بیا با من ببین,Bia Ba Man Bebin,1800,NaN,3.38,92718,978-9648148282,149,1396,NaN,...,"[ ادبیات معاصر , شعر , ادبیات ایران , شعر ا...",رقعی,شومیز,[مینا وداد],[nan],92718,[nan],[nan],"[105, 294, 437, 510, 511, 565]",[21]
0,استخراج طلا از وب,Web Design,148000,25,3.26,84551,978-6229677919,200,1400,NaN,...,"[ تکنولوژی , ادبیات ایران , طراحی ]",رقعی,شومیز,[حسین قاسم زاده],[nan],84551,[21865],[nan],"[336, 437, 1159]",[2258]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,شاد و دور از استرس,Helping Young People to Beat Stress,5700,NaN,3.35,92064,978-6001660337,200,1390,2005,...,"[ ادبیات استرالیا , دهه 2000 میلادی , روانشن...",رقعی,شومیز,[سارا مک نامارا],[فرناز توانایان فرد],92064,[49220],[4292],"[100, 168, 327, 377]",[3071]
0,آقا گرگه پیاده رفتن را دوست ندارد,The Wolf Who Didn't Want to Walk Anymore,35000,30,3.01,47719,978-6004132367,36,1399,2012,...,"[ ادبیات فرانسه , داستان فانتزی , ادبیات داس...",خشتی,شومیز,[اریان لالمان],[مهناز عسگری],47719,[26950],[19730],"[22, 27, 103, 114, 167, 504, 539, 823]",[1326]
0,اجسام,things,10000,NaN,4.0,14713,978-6003043602,88,1399,NaN,...,"[ ادبیات معاصر , دهه 2010 میلادی , شعر , اد...",رقعی,شومیز,[بابک زمانی],[nan],14713,[3272],[nan],"[105, 167, 294, 437, 510, 565, 566]",[1334]
0,آشنایی با وسایل خانه,Ashnaee Ba Vasayele Khane,18500,NaN,3.28,78943,978-6006518152,12,1400,NaN,...,"[ ادبیات کودک و نوجوان , ادبیات ایران , کتاب...",خشتی,شومیز,[سمیه قدیری],[nan],78943,[43203],[nan],"[114, 437, 504, 539, 590, 865]",[1403]


In [228]:
df_3.to_csv('final_file_4000.csv')

In [229]:
df_new = pd.read_csv('final_file_4000.csv').drop(['Unnamed: 0'],axis=1)

In [230]:
df_new

,title_persian,title_english,price,discount,grade,code,shabak,count_page,realese_year_sh,release_year_mi,...,category,ghate,cover,writer,translator,book_id,writer_id,translator_id,category_id,publisher_id
0,همسفر زندگی,Mutluluğu Sende Bulan Senindir Ötesi Misafir,119900,20.0,3.36,110080.0,978-6222208806,160.0,1402.0,2023.0,...,"[' ادبیات ترکیه ', ' روانشناسی ', ' دهه 2020 م...",رقعی,شومیز,['هاکان منگوچ'],['فائزه پورعلی'],110080,[13315],[13316],"['151', '327', '801', '1291']",[1153]
1,منطق حیرانی,Mantegh-e Heyrani,170000,NaN,3.59,32953.0,978-6004901673,304.0,1401.0,NaN,...,"[' ادبیات واقع گرایانه ', ' ادبیات ایران ', ' ...",رقعی,شومیز,['حسن جعفری تبار'],[nan],32953,[15821],[nan],"['139', '437', '591']",[47]
2,مذهب علیه مذهب,Religion Vs. Religion,130000,30.0,3.79,4544.0,978-6005957884,392.0,1400.0,1971.0,...,"[' ادبیات معاصر ', ' دهه 1970 میلادی ', ' فلسف...",رقعی,شومیز,['علی شریعتی'],[nan],4544,[4324],[nan],"['105', '171', '291', '353', '437']",[1492]
3,بیا با من ببین,Bia Ba Man Bebin,1800,NaN,3.38,92718.0,978-9648148282,149.0,1396.0,NaN,...,"[' ادبیات معاصر ', ' شعر ', ' ادبیات ایران ', ...",رقعی,شومیز,['مینا وداد'],[nan],92718,[nan],[nan],"['105', '294', '437', '510', '511', '565']",[21]
4,استخراج طلا از وب,Web Design,148000,25.0,3.26,84551.0,978-6229677919,200.0,1400.0,NaN,...,"[' تکنولوژی ', ' ادبیات ایران ', ' طراحی ']",رقعی,شومیز,['حسین قاسم زاده'],[nan],84551,[21865],[nan],"['336', '437', '1159']",[2258]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,شاد و دور از استرس,Helping Young People to Beat Stress,5700,NaN,3.35,92064.0,978-6001660337,200.0,1390.0,2005.0,...,"[' ادبیات استرالیا ', ' دهه 2000 میلادی ', ' ر...",رقعی,شومیز,['سارا مک نامارا'],['فرناز توانایان فرد'],92064,[49220],[4292],"['100', '168', '327', '377']",[3071]
4544,آقا گرگه پیاده رفتن را دوست ندارد,The Wolf Who Didn't Want to Walk Anymore,35000,30.0,3.01,47719.0,978-6004132367,36.0,1399.0,2012.0,...,"[' ادبیات فرانسه ', ' داستان فانتزی ', ' ادبیا...",خشتی,شومیز,['اریان لالمان'],['مهناز عسگری'],47719,[26950],[19730],"['22', '27', '103', '114', '167', '504', '539'...",[1326]
4545,اجسام,things,10000,NaN,4.00,14713.0,978-6003043602,88.0,1399.0,NaN,...,"[' ادبیات معاصر ', ' دهه 2010 میلادی ', ' شعر ...",رقعی,شومیز,['بابک زمانی'],[nan],14713,[3272],[nan],"['105', '167', '294', '437', '510', '565', '566']",[1334]
4546,آشنایی با وسایل خانه,Ashnaee Ba Vasayele Khane,18500,NaN,3.28,78943.0,978-6006518152,12.0,1400.0,NaN,...,"[' ادبیات کودک و نوجوان ', ' ادبیات ایران ', '...",خشتی,شومیز,['سمیه قدیری'],[nan],78943,[43203],[nan],"['114', '437', '504', '539', '590', '865']",[1403]
